In [37]:
import re
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
# Load and preprocess sonnet data
sonnet_data = ""
with open("/content/Sonnet.txt", "r") as file:
    sonnet_data = file.read()

sonnets = sonnet_data.split("\n\n")  # Split sonnets based on double lines


In [3]:
# Load and preprocess play data
play_data = ""
with open("/content/alllines.txt", "r") as file:
    play_data = file.read()

play_dialogues = play_data.split("\n")  # Split dialogues based on new lines


In [4]:
def remove_words_with_symbols(text):
    '''
    Removes words with symbols from text
    '''
    pattern = r'\b\w*[^a-zA-Z0-9\s]\w*\b'
    clean_text = re.sub(pattern, '', text)
    return clean_text

def remove_inverted_commas(text):
    '''
    Removes inverted commas from text
    '''
    pattern = r'^"|"$'
    clean_text = re.sub(pattern, '', text, flags=re.MULTILINE)
    return clean_text

def remove_punctuation(text):
    '''
    Removes punctuation from text
    '''
    pattern = r'[^\w\s]'
    clean_text = re.sub(pattern, '', text)
    return clean_text

def convert_to_lower(text):
    '''
    Converts text to lower case
    '''
    return text.lower()

In [5]:
for i in range(len(sonnets)):
    sonnets[i] = remove_words_with_symbols(sonnets[i])

In [6]:
for i in range(len(play_dialogues)):
    play_dialogues[i] = remove_inverted_commas(play_dialogues[i])

In [7]:
for i in range(len(play_dialogues)):
    play_dialogues[i] = remove_punctuation(play_dialogues[i])

for i in range(len(sonnets)):
    sonnets[i] = remove_punctuation(sonnets[i])

In [8]:
for i in range(len(play_dialogues)):
    play_dialogues[i] = convert_to_lower(play_dialogues[i])

for i in range(len(sonnets)):
    sonnets[i] = convert_to_lower(sonnets[i])

In [9]:
# Combine sonnet and play data
combined_data = sonnets + play_dialogues

In [10]:
# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(combined_data)
total_words = len(tokenizer.word_index) + 1

In [11]:
# Generate input sequences
input_sequences = []
for line in combined_data:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [12]:
# Pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')

In [13]:
# Split input sequences into input and output
input_sequences = tf.keras.preprocessing.sequence.pad_sequences(input_sequences)
xs, labels = input_sequences[:, :-1], input_sequences[:, -1]

In [14]:
# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_words, 100, input_length=max_sequence_len-1),
    # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150, return_sequences=True)),
    tf.keras.layers.LSTM(150, return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(100),
    tf.keras.layers.Dense(total_words/2, activation='relu'),
    tf.keras.layers.Dense(total_words, activation='softmax')
])

In [15]:
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
# Train the model
history = model.fit(xs, labels, epochs=10)

Epoch 1/10
15608/15608 [==============================] - 1199s 76ms/step - loss: 6.6089 - accuracy: 0.0633
Epoch 2/10
15608/15608 [==============================] - 1154s 74ms/step - loss: 6.2089 - accuracy: 0.0919
Epoch 3/10
15608/15608 [==============================] - 1150s 74ms/step - loss: 6.0274 - accuracy: 0.1010
Epoch 4/10
15608/15608 [==============================] - 1150s 74ms/step - loss: 5.9056 - accuracy: 0.1074
Epoch 5/10
15608/15608 [==============================] - 1148s 74ms/step - loss: 5.8137 - accuracy: 0.1124
Epoch 6/10
15608/15608 [==============================] - 1147s 73ms/step - loss: 5.7372 - accuracy: 0.1160
Epoch 7/10
15608/15608 [==============================] - 1148s 74ms/step - loss: 5.6663 - accuracy: 0.1196
Epoch 8/10
15608/15608 [==============================] - 1147s 73ms/step - loss: 5.6001 - accuracy: 0.1231
Epoch 9/10
15608/15608 [==============================] - 1147s 73ms/step - loss: 5.5355 - accuracy: 0.1257
Epoch 10/10
15608/15608 [===

In [33]:
# Generate a sonnet
seed_text = "the love was flowered"
next_words = 10  # Number of words to generate in the sonnet

In [34]:
for _ in range(11 - len(seed_text.split())):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

1/1 [==============================] - 0s 34ms/step


In [35]:
print(seed_text)

the love was flowered not a man of the king of


In [36]:
model.save('new_model.h5')

In [38]:
# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [39]:
# loading
with open('/content/tokenizer.pickle', 'rb') as handle:
    tokenizerLoaded = pickle.load(handle)

In [40]:
max_sequence_len

163

In [93]:
def generate_sonnet(seed_text):
  MAX_SEQ_LEN = 163
  for i in range(14):
    token_list = tokenizerLoaded.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen = MAX_SEQ_LEN-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""

    for word, index in tokenizer.word_index.items():
      if index == predicted:
        output_word = word
        break
    seed_text += " " + output_word
  return seed_text

In [98]:
pred1 = generate_sonnet("loving beyond")

1/1 [==============================] - 0s 25ms/step


In [99]:
pred

'love to the world and i will tell you what a man is not a'

In [100]:
pred1

'loving beyond the world and i will be a little of the world and i am'